# Guam Forest Inventory Survey
This iPython notebook creates an SQLite database containing US Forest Service Forest Inventory data collected on Guam in 2003 and 2013.

The Forest Service has kindly provided online access to data from its national forest inventory surveys on its [FIA DataMart](http://apps.fs.fed.us/fiadb-downloads/datamart.html) page. Data are available as Microsoft Access databases (\*.accx) for Windows users and as plain text tables (\*.csv) for Mac and Linux users.

Survey methods and resulting data are very thoroughly documented in PDFs stored in the Access database zip file. For Guam, this is http://apps.fs.fed.us/fiadb-downloads/Databases/GUaccdb.zip. 

### Import libraries used in this script

In [25]:
import zipfile  
import glob
import pandas as pd
import sqlite3
import os
import wget

### Download zip files from US Forest Service FIA site
Here we download two zip files, one containing Guam survey data, the other containing reference tables.

In [26]:
!wget 'http://apps.fs.fed.us/fiadb-downloads/GU.zip'
!wget 'http://apps.fs.fed.us/fiadb-downloads/FIADB_REFERENCE.zip'

--2016-03-25 18:45:26--  http://apps.fs.fed.us/fiadb-downloads/GU.zip
Resolving apps.fs.fed.us (apps.fs.fed.us)... 165.221.108.16
Connecting to apps.fs.fed.us (apps.fs.fed.us)|165.221.108.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 320149 (313K) [application/x-zip-compressed]
Saving to: ‘GU.zip’

100%[======================================>] 320,149     53.1KB/s   in 7.4s   

2016-03-25 18:45:35 (42.4 KB/s) - ‘GU.zip’ saved [320149/320149]

--2016-03-25 18:45:35--  http://apps.fs.fed.us/fiadb-downloads/FIADB_REFERENCE.zip
Resolving apps.fs.fed.us (apps.fs.fed.us)... 165.221.108.16
Connecting to apps.fs.fed.us (apps.fs.fed.us)|165.221.108.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5243888 (5.0M) [application/x-zip-compressed]
Saving to: ‘FIADB_REFERENCE.zip’

100%[======================================>] 5,243,888    111KB/s   in 63s    

2016-03-25 18:46:39 (81.1 KB/s) - ‘FIADB_REFERENCE.zip’ saved [5243888/5243888]



### Extract the zip files

In [27]:
zipfile.ZipFile('GU.zip').extractall()
zipfile.ZipFile('FIADB_REFERENCE.zip').extractall()

### Create a dict containing the number of lines in each csv file

In [28]:
def line_count(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

line_count_dict = {}
for fname in glob.glob('*.csv'):
    line_count_dict[fname] = line_count(fname)

sorted(line_count_dict.items(), key=lambda x: x[1], reverse=True)

[('REF_PLANT_DICTIONARY.csv', 78892),
 ('REF_HABTYP_DESCRIPTION.csv', 8711),
 ('GU_P2VEG_SUBP_STRUCTURE.csv', 5305),
 ('REF_SPECIES.csv', 2592),
 ('LICHEN_SPECIES_SUMMARY.csv', 2405),
 ('GU_TREE_REGIONAL_BIOMASS.csv', 2353),
 ('GU_TREE.csv', 2353),
 ('REF_INVASIVE_SPECIES.csv', 1641),
 ('GU_SUBP_COND.csv', 1548),
 ('GU_SUBPLOT.csv', 1481),
 ('GU_P2VEG_SUBPLOT_SPP.csv', 1052),
 ('REF_LICHEN_SPECIES.csv', 987),
 ('GU_POP_PLOT_STRATUM_ASSGN.csv', 739),
 ('GU_INVASIVE_SUBPLOT_SPP.csv', 464),
 ('GU_COND.csv', 444),
 ('GU_PLOTGEOM.csv', 371),
 ('GU_PLOTSNAP.csv', 371),
 ('GU_PLOT.csv', 371),
 ('GU_SEEDLING.csv', 363),
 ('REF_LICHEN_SPP_COMMENTS.csv', 211),
 ('REF_UNIT.csv', 210),
 ('REF_FOREST_TYPE.csv', 208),
 ('GU_POP_EVAL_ATTRIBUTE.csv', 151),
 ('REF_POP_ATTRIBUTE.csv', 135),
 ('REF_HABTYP_PUBLICATION.csv', 126),
 ('GU_BOUNDARY.csv', 91),
 ('REF_RESEARCH_STATION.csv', 60),
 ('REF_STATE_ELEV.csv', 59),
 ('REF_SPECIES_GROUP.csv', 55),
 ('REF_CITATION.csv', 36),
 ('REF_FOREST_TYPE_GROUP.csv'

### Import the csv files as tables in an SQLite database
CSV files containing no data (line_count = 1) are not imported.The first column of each table is indexed.

In [29]:
conn = sqlite3.connect('guam_forest_survey.db')

for fname, line_count in line_count_dict.iteritems():
    if line_count > 1:
        print fname, line_count
        df = pd.read_csv(fname, index_col=0, low_memory=False)
        df.to_sql(con=conn, name=fname.replace('.csv', ''), if_exists='replace')        

GU_POP_EVAL.csv 5
GU_SURVEY.csv 3
REF_PLANT_DICTIONARY.csv 78892
REF_POP_ATTRIBUTE.csv 135
GU_P2VEG_SUBPLOT_SPP.csv 1052
REF_CITATION.csv 36
REF_HABTYP_DESCRIPTION.csv 8711
GU_SUBPLOT.csv 1481
GU_POP_PLOT_STRATUM_ASSGN.csv 739
GU_PLOTGEOM.csv 371
GU_POP_ESTN_UNIT.csv 5
GU_TREE_REGIONAL_BIOMASS.csv 2353
REF_STATE_ELEV.csv 59
REF_LICHEN_SPP_COMMENTS.csv 211
GU_POP_EVAL_TYP.csv 7
REF_HABTYP_PUBLICATION.csv 126
GU_SUBP_COND.csv 1548
REF_FOREST_TYPE_GROUP.csv 35
GU_POP_EVAL_GRP.csv 3
REF_SPECIES.csv 2592
GU_SEEDLING.csv 363
GU_COND.csv 444
REF_POP_EVAL_TYP_DESCR.csv 10
REF_INVASIVE_SPECIES.csv 1641
GU_PLOTSNAP.csv 371
REF_SPECIES_GROUP.csv 55
GU_P2VEG_SUBP_STRUCTURE.csv 5305
GU_BOUNDARY.csv 91
GU_POP_EVAL_ATTRIBUTE.csv 151
GU_POP_STRATUM.csv 21
BEGINEND.csv 3
LICHEN_SPECIES_SUMMARY.csv 2405
REF_FIADB_VERSION.csv 20
GU_INVASIVE_SUBPLOT_SPP.csv 464
GU_TREE.csv 2353
REF_RESEARCH_STATION.csv 60
GU_COUNTY.csv 2
REF_FOREST_TYPE.csv 208
GU_PLOT.csv 371
REF_LICHEN_SPECIES.csv 987
REF_UNIT.csv 210


### Test Query
Creates a dataframe containing the number of trees (DBH >= 5 inches) observed in plots during the 2013 survey.

In [30]:
sql = '''
select gu_tree.SPCD, GENUS, SPECIES,  COMMON_NAME, count(*) as N
from gu_tree, ref_species
where INVYR = 2013
    and gu_tree.SPCD = ref_species.SPCD
group by gu_tree.SPCD
order by N desc;
'''
df = pd.read_sql(con=conn, sql=sql)
df

,SPCD,GENUS,SPECIES,COMMON_NAME,N
0,7412,Hibiscus,tiliaceus,sea hibiscus,154
1,8874,Vitex,parviflora,smallflower chastetree,153
2,908,Cocos,nucifera,coconut palm,115
3,7565,Leucaena,leucocephala,white leadtree,114
4,7377,Heterospathe,elata,palma brava,99
5,8085,Pandanus,tectorius,Tahitian screwpine,64
6,8307,Premna,obtusifolia,ahgao,62
7,7282,Guamia,mariannae,paipai,61
8,8842,Triphasia,trifolia,limeberry,55
9,6852,Cycas,circinalis,queen sago,46


### Garbage Collection
Delete CSV and ZIP files.

In [31]:
for fname in glob.glob('*.csv'):
  os.remove(fname)
for fname in glob.glob('*.zip'):
  os.remove(fname)